In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
import PIL
import matplotlib.pyplot as plt
import os
from pathlib import Path
import shutil
from progressbar import progressbar
import time
import PIL
from birdbox.database import Image, Bird
from birdbox.tools import detection_box_to_crop_box, visualize_detection_boxes
from birdbox.detection import Detection, detect_and_classify_birds, load_detector
from birdbox.classifiers import Classifier
from yolov4deepsort.deep_sort.tracker import Tracker
from yolov4deepsort.deep_sort.detection import Detection as ds_Detection
from yolov4deepsort.tools import generate_detections
from yolov4deepsort.deep_sort import nn_matching

# DETECTOR_MODEL_NAME = "centernet_hg104_1024x1024_coco17_tpu-32"
DETECTOR_MODEL_NAME = "ssd_mobilenet_v2_320x320_coco17_tpu-8"
CLASSIFIER_NAME = "EfficientNetB0_120x120_1-3-4-5-6-7-8-9-10-11-12"

In [3]:
[detect_fn, category_index] = load_detector(model_name=DETECTOR_MODEL_NAME)

Loading model...Done! Took 33.64980459213257s


In [4]:
classifier = Classifier.load(CLASSIFIER_NAME)

In [5]:
classifier.validation_accuracy()

0.9173553586006165

In [12]:
# VIDEO_FILE, FACTOR, SKIP = r'videos\blut_tit_vs_chaffinch.mp4', 1, 3
VIDEO_FILE, FACTOR, SKIP = r'videos\youtube4.mp4', 1, 1
# VIDEO_FILE, FACTOR, SKIP = r"videos\futter.mp4", 2, 1
# VIDEO_FILE, FACTOR, SKIP = r'videos\woodland.mp4', 2, 1
# VIDEO_FILE, FACTOR, SKIP = r'videos\woodland_Amsel.mp4', 2, 1
# VIDEO_FILE, FACTOR, SKIP = r'videos\forest_woodpecker.mp4', 1, 1

In [13]:
max_cosine_distance = 0.4
nn_budget = None
model_filename = r'yolov4deepsort\model_data\mars-small128.pb'
encoder = generate_detections.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

cap = cv2.VideoCapture(VIDEO_FILE)

counter = 0
cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

In [14]:
try:
    while cap.isOpened():
        
        ret, frame = cap.read()    
        if not ret:
            break
        counter += 1
        if counter % (SKIP+1) != 0:
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = detect_and_classify_birds(frame_rgb, detect_fn, classifier.model, classifier.image_size, min_score=0.4,
                                              max_intersection_over_size=0.001)        
        
        frame_size = frame_rgb.shape[1], frame_rgb.shape[0]

        boxes_ltwh = [detection_box_to_crop_box(d.box, image_size=frame_size).left_top_width_height() for d in detections]
        scores = [d.score for d in detections]
        
        features = encoder(frame, boxes_ltwh)
        detections = [ds_Detection(bbox, score, "Bird", feature) for bbox, score, feature in zip(boxes_ltwh, scores, features)]        
        
        tracker.predict()
        tracker.update(detections)
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue 
            bbox = track.to_tlbr()
            class_name = track.get_class()
        
            color = colors[int(track.track_id) % len(colors)]
            color = [i * 255 for i in color]
            cv2.rectangle(frame_rgb, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
            cv2.rectangle(frame_rgb, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
            cv2.putText(frame_rgb, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)

        result = np.asarray(frame_rgb)
        result = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        cv2.imshow("Output Video", result)
            
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
except Exception as e:
    print(e)
finally:
    cap.release()
    cv2.destroyAllWindows()